# Data

In [ ]:
import torch
import torch.nn as nn
import torchvision.models as models
import torchvision.transforms as transforms
from configparser import ConfigParser
from tqdm import tqdm
import re
import time
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
import textwrap
import os
import glob
import random
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import shutil
from openai import OpenAI
import os
import base64
import json
from io import BytesIO
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, accuracy_score, classification_report,precision_recall_fscore_support
import json
from scipy.stats import wasserstein_distance
from scipy.spatial.distance import euclidean
from scipy.linalg import norm
from scipy.stats import energy_distance
from scipy.spatial.distance import mahalanobis
from scipy.spatial.distance import braycurtis
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import (
    confusion_matrix,
    ConfusionMatrixDisplay,
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
)

random.seed(42)

In [ ]:
train_dir = "path_to_train_directory""
test_dir =  "path_to_test_directory"

In [ ]:
def get_data(root_directory):
    image_text_pairs = []
    # for label in os.listdir(root_directory):
    for label in ["lesion","normal","variation in normal","red lesion"]:
      label_dir = os.path.join(root_directory, label)
      if os.path.isdir(label_dir):
        # Iterate through each image in the label directory
        for image_file in os.listdir(label_dir):
            image_path = os.path.join(label_dir, image_file)
            # Check if the path is a file (to avoid subdirectories)
            if os.path.isfile(image_path):
                # Add the image path and label to the list
                if label != 'normal':
                    image_text_pairs.append((image_path, 'lesion'))
                    continue
                image_text_pairs.append((image_path, label))
    return image_text_pairs

In [ ]:
test_df = pd.DataFrame(get_data(test_dir), columns=['image path','label'])
test_df

In [ ]:
px.histogram(test_df, y="label", color="label", title="Classes Distribution")

In [ ]:
train_df = pd.DataFrame(get_data(train_dir), columns=['image path','label'])
train_df.shape

In [ ]:
test_label_counts = test_df['label'].value_counts()
train_label_counts = train_df['label'].value_counts()

In [ ]:
print(test_label_counts)
print(train_label_counts)

# Retriever

In [ ]:
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"

def get_image_encoder(image_encoder):
    if torch.cuda.is_available():
        device = torch.device("cuda")
    else:
        device = torch.device("cpu")
    print(f"Using device: {device}")
    if image_encoder == "resnet50_pretrained":
        resnet50 = models.resnet50(pretrained=True)
        resnet50 = models.resnet50(pretrained=True)
        resnet50 = torch.nn.Sequential(*list(resnet50.children())[:-1])  # Remove classification layer
        resnet50.eval()
        resnet50.to(device)
        return resnet50
    elif image_encoder == "resnet50_finetuned":
        resnet50 = models.resnet50(pretrained=False)
        for param in resnet50.parameters():
            param.requires_grad = False

        # Replace the final fully connected layer with a new one
        num_features = resnet50.fc.in_features  # Get the number of input features for the fc layer
        resnet50.fc = nn.Linear(num_features, 2)  # New trainable classifier layer
        path = "encoder_path.pth"
        resnet50.load_state_dict(torch.load(path, map_location=device))
        resnet50 = torch.nn.Sequential(*list(resnet50.children())[:-1])  # Remove classification layer
        resnet50.eval()
        return resnet50




def extract_number_from_filename(file_path):
    # Get the file name from the path
    file_name = os.path.basename(file_path)

    # Use regular expressions to find all numbers in the file name
    match = re.search(r'\d+', file_name)
    return int(match.group())


def get_preprocessor():
    preprocess = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
    return preprocess

def get_wasserstein_distance(a, b):
    return np.array([wasserstein_distance(a[i], b) for i in range(len(a))])

def get_euclidean_distance(a, b):
    return np.array([euclidean(a[i], b) for i in range(len(a))])

def get_mahalanobis_distance(a, b, VI=None):
    if VI is None:
        # Estimate covariance from dataset embeddings
        cov = np.cov(a, rowvar=False) + 1e-6 * np.eye(a.shape[1])
        VI = np.linalg.inv(cov)
    return np.array([mahalanobis(a[i], b, VI) for i in range(len(a))])

def get_cosine_similarity(a, b):
    return cosine_similarity(a,b)

def get_energy_distance(a, b):
    return np.array([energy_distance(a[i], b) for i in range(len(a))])

def get_braycurtis_distance(a, b):
    return np.array([braycurtis(a[i], b) for i in range(len(a))])

def get_norm(a,b,ord):
    output_list = []
    # to traverse rows
    for i in range(len(a)):
      a_vec = np.array(a[i])
      input = []
      for j in range(len(a[i])):
          input.append(a_vec[j]-b[j])
      norm_value = norm(input, ord = ord)
      output_list.append(norm_value)
    return np.array(output_list)


class Retriever:
    def __init__(self,dataset,image_encoder,vector_comparision_metric):
        # Load the pre-trained ResNet50 model without the top layer (for feature extraction)
        self.dataset = dataset
        self.image_encoder = image_encoder
        self.vector_comparision_metric = vector_comparision_metric
        self.model = get_image_encoder(image_encoder)
        self.preprocessor = get_preprocessor()
        self.lesion_embeddings_df = None
        self.lesion_paths = []
        self.normal_embeddings_df = None
        self.normal_paths = []
        self.index_lesion = None
        self.index_normal = None
        self.top_5_lesion_df = []
        self.top_5_normal_df = []
        self.normal_embeddings_path = "normal_embeddings.pkl"
        self.lesion_embeddings_path = "lesion_embeddings.pkl"
        self.norm_ord_dict = {
            "one":1,
            "two":2,
            "inf":np.inf,
            "-inf":-np.inf
        }
        self.dimension = None

    def extract_embedding(self,image_path):
      img = Image.open(image_path)
      img = self.preprocessor(img).unsqueeze(0)  # Add batch dimension
      with torch.no_grad():
          embedding = self.model(img)
      return embedding.squeeze().numpy()

    def generate_embeddings(self,label):
        embeddings = []
        label_df = self.dataset[self.dataset["label"]==label]
        label_df.reset_index(inplace=True)
        print("Generating embdeddibgs of "+label+" images....")
        # print(label_df.index)
        # print(label_df.shape)
        for idx in tqdm(label_df.index):
            # print(idx)
            img_path = label_df.iloc[idx]["image path"]
            embedding = self.extract_embedding(img_path)
            embeddings.append(np.array(embedding))
        label_df["embeddings"] = embeddings
        return label_df


    def load_ebmbeddings(self):
        # Check and load or generate lesion embeddings
        if os.path.exists(self.lesion_embeddings_path):
            print("Lesion embeddings file found. Loading embeddings...")
            self.lesion_embeddings_df = pd.read_pickle(self.lesion_embeddings_path)
        else:
            print("Lesion embeddings file not found. Generating embeddings...")
            self.lesion_embeddings_df = self.generate_embeddings("lesion")
            self.lesion_embeddings_df.to_pickle(self.lesion_embeddings_path)

        # Check and load or generate normal embeddings
        if os.path.exists(self.normal_embeddings_path):
            print("Normal embeddings file found. Loading embeddings...")
            self.normal_embeddings_df = pd.read_pickle(self.normal_embeddings_path)
        else:
            print("Normal embeddings file not found. Generating embeddings...")
            self.normal_embeddings_df = self.generate_embeddings("normal")
            self.normal_embeddings_df.to_pickle(self.normal_embeddings_path)

    # Function to perform a similarity search and retrieve top 10 images
    def retrieve_top_k_images(self, query_image_path, embeddings_df,k=5):
        image_paths = embeddings_df["image path"]

        embeddings = np.vstack(embeddings_df["embeddings"].values)

        query_embedding = self.extract_embedding(query_image_path)


        if self.vector_comparision_metric == "cosine":
            query_embedding = np.expand_dims(query_embedding, axis=0)
            similarities = get_cosine_similarity(query_embedding,embeddings)
            top_k_indices = np.argsort(similarities[0])[::-1][:k]  # Sort in ascending order
            # Retrieve the corresponding image paths and similarities
            top_k_images = [image_paths[i] for i in top_k_indices]
            top_k_similarities = [similarities[0][i] for i in top_k_indices]

            top_k_dict = {
                "top k images":top_k_images,
                "top_k_similarities": top_k_similarities
            }
            top_k_df=  pd.DataFrame(top_k_dict)
            return top_k_df

        elif self.vector_comparision_metric in self.norm_ord_dict.keys():
            similarities = get_norm(embeddings, query_embedding,self.norm_ord_dict[self.vector_comparision_metric])
            top_k_indices = np.argsort(similarities)[:k]  # Sort in ascending order
            # Retrieve the corresponding image paths and similarities
            top_k_images = [image_paths[i] for i in top_k_indices]
            top_k_similarities = [similarities[i] for i in top_k_indices]

            top_k_dict = {
                "top k images":top_k_images,
                "top_k_similarities": top_k_similarities
            }
            top_k_df=  pd.DataFrame(top_k_dict)
            return top_k_df

        elif self.vector_comparision_metric == "wasserstein":
            similarities = get_wasserstein_distance(embeddings, query_embedding)
            top_k_indices = np.argsort(similarities)[:k]  # Sort in ascending order
            # Retrieve the corresponding image paths and similarities
            top_k_images = [image_paths[i] for i in top_k_indices]
            top_k_similarities = [similarities[i] for i in top_k_indices]

            top_k_dict = {
                "top k images":top_k_images,
                "top_k_similarities": top_k_similarities
            }
            top_k_df=  pd.DataFrame(top_k_dict)
            return top_k_df


        elif self.vector_comparision_metric == "braycurtis":
            similarities = get_braycurtis_distance(embeddings, query_embedding)
            top_k_indices = np.argsort(similarities)[:k]  # smaller = more similar
            top_k_images = [image_paths[i] for i in top_k_indices]
            top_k_similarities = [similarities[i] for i in top_k_indices]

            top_k_dict = {
                "top k images": top_k_images,
                "top_k_similarities": top_k_similarities
            }
            top_k_df = pd.DataFrame(top_k_dict)
            return top_k_df


        elif self.vector_comparision_metric == "euclidean":
            similarities = get_euclidean_distance(embeddings, query_embedding)
            top_k_indices = np.argsort(similarities)[:k]  # Sort in ascending order
            # Retrieve the corresponding image paths and similarities
            top_k_images = [image_paths[i] for i in top_k_indices]
            top_k_similarities = [similarities[i] for i in top_k_indices]

            top_k_dict = {
                "top k images":top_k_images,
                "top_k_similarities": top_k_similarities
            }
            top_k_df=  pd.DataFrame(top_k_dict)
            return top_k_df

        elif self.vector_comparision_metric == "energy":
            similarities = get_energy_distance(embeddings, query_embedding)
            top_k_indices = np.argsort(similarities)[:k]  # Sort in ascending order
            # Retrieve the corresponding image paths and similarities
            top_k_images = [image_paths[i] for i in top_k_indices]
            top_k_similarities = [similarities[i] for i in top_k_indices]

            top_k_dict = {
                "top k images":top_k_images,
                "top_k_similarities": top_k_similarities
            }
            top_k_df=  pd.DataFrame(top_k_dict)
            return top_k_df

        elif self.vector_comparision_metric == "mahalanobis":
            similarities = get_mahalanobis_distance(embeddings, query_embedding)
            top_k_indices = np.argsort(similarities)[:k]  # smaller = more similar
            top_k_images = [image_paths[i] for i in top_k_indices]
            top_k_similarities = [similarities[i] for i in top_k_indices]

            top_k_dict = {
                "top k images": top_k_images,
                "top_k_similarities": top_k_similarities
            }
            top_k_df = pd.DataFrame(top_k_dict)
            return top_k_df


        elif self.vector_comparision_metric == "random":
            top_k_indices = np.random.choice(len(embeddings_df), k, replace=False)
            top_k_images = [image_paths[i] for i in top_k_indices]
            top_k_similarities = ['random' for i in top_k_indices]

            top_k_dict = {
                "top k images":top_k_images,
                "top_k_similarities": top_k_similarities
            }
            top_k_df=  pd.DataFrame(top_k_dict)
            return top_k_df

    # Retrieve top 5 images from Class A and Class B for a given query image
    def retrieve_images(self, query_image_path):
        self.top_5_lesion_df = self.retrieve_top_k_images(query_image_path, self.lesion_embeddings_df, k=5)
        self.top_5_normal_df = self.retrieve_top_k_images(query_image_path, self.normal_embeddings_df, k=5)
        return query_image_path, self.top_5_lesion_df, self.top_5_normal_df

# Data Processor

In [ ]:
def to_base64(path):
    with open(path, 'rb') as image_file:
        image_base64 = base64.b64encode(image_file.read()).decode('utf-8')
    return image_base64

class Processor:
    def __init__(self, lesion_image_list,normal_image_list, test_image_path,normal_cot_dict,lesion_cot_dict,with_caution = False, max_size=200):
        if normal_image_list is None:
            normal_image_list = []
        self.lesion_image_list = lesion_image_list
        self.normal_image_list = normal_image_list
        self.normal_cot_dict = normal_cot_dict
        self.lesion_cot_dict = lesion_cot_dict
        self.max_size = max_size
        self.test_image = test_image_path
        self.with_caution = with_caution
        self.messages = None

    def get_image_data(self, img_path):
        return self.resize_and_convert_to_base64(img_path)
    def get_message(self):
        self.messages = [
        {"role": "user", "content": self.get_content()},
        {"role": "user", "content": [
            {
                "type": "image_url",
                "image_url": {
                    "url": f"data:image/jpg;base64,{self.get_image_data(self.test_image)}"
                }
            }]
          }
    ]

    def resize_and_convert_to_base64(self, image_path):
        size = self.max_size
        # Open the image file
        with Image.open(image_path) as img:
            # Check the size of the image
            width, height = img.size

            # Resize if either dimension is greater than 800 pixels
            if width > size or height > size:
                # Determine the new size while maintaining the aspect ratio
                if width > height:
                    new_width = size
                    new_height = int((size / width) * height)
                else:
                    new_height = size
                    new_width = int((size / height) * width)

                # Resize the image
                img = img.resize((new_width, new_height), Image.Resampling.LANCZOS)

            # Convert the image to base64
            buffered = BytesIO()
            img.save(buffered, format="PNG")
            img_base64 = base64.b64encode(buffered.getvalue()).decode("utf-8")

            return img_base64

    def get_content(self):
        main_prompt = {
            "type": "text",
            "text":"""Here are some images and their respective thought process and reasons to classify them into their output classes.
                      Please act as an image classifier while following a similar thought process and classify the last image as either "abnormal" or "normal".
                      Your response should be a JSON object with the following keys:
                      - "thought"
                      - "output"
                        """
        }
        main_prompt_with_caution ={
            "type": "text",
            "text":"""Here are some images and their respective thought process
                      and reasons to classify them into their output classes.
                      Please act as an image classifier while following a similar
                      thought process and classify the last image as either "abnormal" or "normal".
                      Caution on the side of the abnormal rather than the normal.
                      Falsely classifying an image as normal isn't okay.
                      Your response should be a JSON object with the following keys:
                      - "thought"
                      - "output"
                        """
          }

        content_list = [main_prompt_with_caution if self.with_caution else main_prompt]

        for img_path in self.lesion_image_list:
            content_list.append(
                {
                    "type": "image_url",
                    "image_url": {
                        "url": f"data:image/jpg;base64,{self.get_image_data(img_path)}"
                    }
                }
            )

            content_list.append(
                {"type": "text", "text": self.lesion_cot_dict[img_path]}
            )
            content_list.append(
                {"type": "text", "text": "Output:abnormal"}
            )
        for img_path in self.normal_image_list:
            content_list.append(
                {
                    "type": "image_url",
                    "image_url": {
                        "url": f"data:image/jpg;base64,{self.get_image_data(img_path)}"
                    }
                }
            )
            content_list.append(
                {"type": "text", "text": self.normal_cot_dict[img_path]}
            )
            content_list.append(
                {"type": "text", "text": "Output:normal"}
            )
        return content_list

### Augmentation and generation

In [ ]:
# load COT dict
def load_cot_dict():
    with open(f"path_to_var_normal_cot_dict", 'r') as file:
        var_normal_cot_dict = json.load(file)

    with open("path_to_red_lesion_cot_dict") as file:
        red_lesion_cot_dict = json.load(file)

    with open(f"path_to_normal_cot_dict", 'r') as file:
        normal_cot_dict = json.load(file)

    with open(f"path_to_lesion_cot_dict", 'r') as file:
        lesion_cot_dict = json.load(file)
        lesion_cot_dict.update(var_normal_cot_dict)
        lesion_cot_dict.update(red_lesion_cot_dict)
    return lesion_cot_dict,normal_cot_dict

In [ ]:
def save_results(results,expert_name):
  file_name = f"test_{expert_name}.json"
  with open(file_name, 'w') as file:
    json.dump(results, file)
  print(f"Results saved to {file_name}")

In [ ]:
def extract_output(prediction):
    # Remove the markdown code block syntax (```json and ```)
    prediction = prediction.strip().replace('```json\n', '').replace('```', '')
    # Parse the JSON string
    prediction_dict = json.loads(prediction)
    # Return the "output" value
    return prediction_dict

# Expert

In [ ]:
class Expert(Retriever,Processor):
    def __init__(self,image_encoder,vector_comparision_metric,run_num,train_df,test_df,with_caution = False):
        self.with_caution = with_caution
        self.expert_name = image_encoder+"_"+vector_comparision_metric+"_with_caution"+"_Run"+run_num if self.with_caution else image_encoder+"_"+vector_comparision_metric+"_Run"+run_num
        self.retriever = Retriever(dataset = train_df,image_encoder = image_encoder,vector_comparision_metric = vector_comparision_metric)
        self.image_encoder = image_encoder
        self.vector_comparision_metric = vector_comparision_metric
        self.client = OpenAI(api_key = "your_api_key")
        self.lesion_cot_dict = {}
        self.normal_cot_dict = {}
        self.results = {}
        self.test_df = test_df

    def load_cot_dict():
        with open(f"path_to_var_normal_cot_dict", 'r') as file:
            var_normal_cot_dict = json.load(file)

        with open("path_to_red_lesion_cot_dict") as file:
            red_lesion_cot_dict = json.load(file)

        with open(f"path_to_normal_cot_dict", 'r') as file:
            normal_cot_dict = json.load(file)

        with open(f"path_to_lesion_cot_dict", 'r') as file:
            lesion_cot_dict = json.load(file)
            lesion_cot_dict.update(var_normal_cot_dict)
            lesion_cot_dict.update(red_lesion_cot_dict)
            return lesion_cot_dict,normal_cot_dict

    def load_ebmbeddings(self):
        self.retriever.load_ebmbeddings()


    def predict(self,lesion_image_list,normal_image_list,test_image_path):
        image_processor = Processor(lesion_image_list,
                                    normal_image_list,
                                    test_image_path,
                                    self.normal_cot_dict,
                                    self.lesion_cot_dict,
                                    with_caution = self.with_caution)
        image_processor.get_message()
        messages = image_processor.messages

        response = self.client.chat.completions.create(
            model='gpt-4o-2024-11-20',
            messages=messages,
            temperature=0.0,
            max_tokens=200
        )

        return response.choices[0].message.content
        # return response
    def get_top_k_image_list(self,test_image_path):
        query_image_path, top_5_lesion_images, top_5_normal_images= self.retriever.retrieve_images(test_image_path)


        lesion_img_list = top_5_lesion_images["top k images"].values
        lesion_imgs = [os.path.join('lesion',os.path.basename(path)) for path in lesion_img_list if 'variation' not in path and 'red' not in path]
        var_imgs = [os.path.join('variation in normal',os.path.basename(path)) for path in lesion_img_list if 'variation' in path]
        red_lesion_imgs = [os.path.join('Red lesions',os.path.basename(path)) for path in lesion_img_list if 'red' in path]
        lesion_img_list = lesion_imgs + var_imgs+red_lesion_imgs


        normal_img_list = top_5_normal_images["top k images"].values
        normal_img_list = [os.path.join('normal',os.path.basename(path)) for path in normal_img_list]
        return lesion_img_list,normal_img_list,test_image_path

    def get_expert_content_list(self):
        random.seed(42)
        run_images = random.sample(range(len(self.test_df.index)),1)
        for idx in run_images:
            expectation = self.test_df["label"][idx]
            if expectation == "lesion":
                expectation = "abnormal"
            test_image_path = self.test_df["image path"][idx]
            lesion_img_list,normal_img_list,test_image_path = self.get_top_k_image_list(test_image_path)
            test_image_name = test_image_path[94:]
            image_processor = Processor(lesion_img_list,
                                        normal_img_list,
                                        test_image_path,
                                        self.normal_cot_dict,
                                        self.lesion_cot_dict,
                                        with_caution = self.with_caution)
        return  image_processor.get_content()

    def get_result(self,test=False,test_size=10):
        results= {}
        random.seed(42)
        run_images = self.test_df.index if not test else random.sample(range(len(self.test_df.index)),test_size)
        # print(run_images)
        for idx in tqdm(run_images[:10]):
            expectation = self.test_df["label"][idx]
            if expectation == "lesion":
                expectation = "abnormal"
            test_image_path = self.test_df["image path"][idx]
            query_image_path, top_5_lesion_images, top_5_normal_images= self.retriever.retrieve_images(test_image_path)


            lesion_img_list = top_5_lesion_images["top k images"].values
            lesion_imgs = [os.path.join('lesion',os.path.basename(path)) for path in lesion_img_list if 'variation' not in path and 'red' not in path]
            var_imgs = [os.path.join('variation in normal',os.path.basename(path)) for path in lesion_img_list if 'variation' in path]
            red_lesion_imgs = [os.path.join('Red lesions',os.path.basename(path)) for path in lesion_img_list if 'red' in path]
            lesion_img_list = lesion_imgs + var_imgs+red_lesion_imgs


            normal_img_list = top_5_normal_images["top k images"].values
            normal_img_list = [os.path.join('normal',os.path.basename(path)) for path in normal_img_list]

            test_image_name = query_image_path[94:]
            prediction = self.predict(
                                  lesion_img_list,
                                  normal_img_list,
                                  query_image_path
                                )

            pred_dict = extract_output(prediction)
            print(pred_dict)
            results[test_image_name] = {
                "query_image_path": query_image_path,
                "expectation": expectation,
                "prediction": pred_dict["output"],
                "thought": pred_dict["thought"]
            }
            # results[test_image_name] = {
            #     "response": prediction
            # }

        self.results = results
    def save_results(self):
        save_results(self.results,self.expert_name)

# Expert 7
## RAG: Resnet50(pre trained weights) + Braycurtis

In [ ]:
 num_runs = 1
for run in range(1,num_runs+1):
  expert1 = Expert(
      image_encoder = 'resnet50_pretrained',
      vector_comparision_metric = 'inf',
      # run_num = str(run+1),
      run_num = str(run),
      train_df = train_df,
      test_df = test_df,
      with_caution = True
  )
  print(expert1.expert_name)
  expert1.load_ebmbeddings()
  expert1.load_cot_dict()
  expert1.get_result(test=False,test_size = 1)
  expert1.save_results()

In [ ]:
# expert1.load_ebmbeddings()
# expert1.load_cot_dict()

In [ ]:
# expert1.get_result(test=False)
# expert1.save_results()

In [ ]:
# expert1.results

# Expert 1
## RAG: Resnet50(pre trained weights) + Cosine similarity

In [ ]:
num_runs = 5
for run in range(1,2):
  expert1 = Expert(
      image_encoder = 'resnet50_pretrained',
      vector_comparision_metric = 'cosine',
      # run_num = str(run+1),
      run_num = 'cost_analysis',
      train_df = train_df,
      test_df = test_df
  )
  print(expert1.expert_name)
  expert1.load_ebmbeddings()
  expert1.load_cot_dict()
  expert1.get_result(test=False,test_size = 1)
  expert1.save_results()

In [ ]:
# expert1.load_ebmbeddings()
# expert1.load_cot_dict()

In [ ]:
# expert1.get_result(test=False)
# expert1.save_results()

In [ ]:
# expert1.results

# Expert 2
## RAG: Resnet50(pre trained weights) + One norm

In [ ]:
expert3 = Expert(
    image_encoder = 'resnet50_pretrained',
    vector_comparision_metric = 'euclidean',
    train_df = train_df,
    test_df = test_df,
    with_caution = True
)

In [ ]:
expert3.expert_name

In [ ]:
expert3.load_ebmbeddings()
expert3.load_cot_dict()

In [ ]:
# random.seed(42)
# random_test_idx = random.sample(range(len(expert3.test_df.index)),1)[0]
# random_test_image = test_df["image path"][random_test_idx]

In [ ]:
expert3.get_result(test=False,test_size=3)
expert3.save_results()

In [ ]:
# expert3.results

# Expert 2
## RAG: Resnet50(pre trained weights) + Eucledean

In [ ]:
expert2 = Expert(
    image_encoder = 'resnet50_pretrained',
    vector_comparision_metric = 'euclidean',
    train_df = train_df,
    test_df = test_df
)

In [ ]:
expert2.load_ebmbeddings()
expert2.load_cot_dict()

In [ ]:
expert2.get_result(test=True,test_size=2)
expert2.save_results()

# Expert 4

# Expert 5
## RAG: Resnet50(pre trained weights) + random
## normal cot

In [ ]:
expert5 = Expert(
    image_encoder = 'resnet50_pretrained',
    vector_comparision_metric = 'random',
    train_df = train_df,
    test_df = test_df
)

In [ ]:
print(expert5.expert_name)

In [ ]:
expert5.load_ebmbeddings()
expert5.load_cot_dict()

In [ ]:
expert5.get_result(test=False,test_size=2)
expert5.save_results()

# Mixture of Experts

In [ ]:
def to_base64(path):
    with open(path, 'rb') as image_file:
        image_base64 = base64.b64encode(image_file.read()).decode('utf-8')
    return image_base64
def load_expert_results(expert_name):
  run_number = "1"
  file_name = f"Results/{expert_name}_Run{run_number}.json"
  with open(file_name, 'r') as file:
    results = json.load(file)
  print(f"Results loaded from {file_name}")
  return results

expert_dict = {
    "without_caution":['resnet50_pretrained_cosine','resnet50_pretrained_one','resnet50_pretrained_euclidean',"resnet50_pretrained_inf","resnet50_pretrained_-inf"],
    "with_caution":['resnet50_pretrained_cosine_with_caution','resnet50_pretrained_one_with_caution','resnet50_pretrained_euclidean_with_caution',"resnet50_pretrained_inf_with_caution","resnet50_pretrained_-inf_with_caution"]
}

class MixtureOfExpertsProcessor:
    def __init__(self,test_image_path,img_name ,with_caution = False,run_num = "1",max_size=200):
        self.run_num = run_num
        self.with_caution = with_caution
        # self.expert_dict = {
        #                     "without_caution":['resnet50_pretrained_cosine','resnet50_pretrained_one','resnet50_pretrained_euclidean',"resnet50_pretrained_inf","resnet50_pretrained_-inf"],
        #                     "with_caution":['resnet50_pretrained_cosine_with_caution','resnet50_pretrained_one_with_caution','resnet50_pretrained_euclidean_with_caution',"resnet50_pretrained_inf_with_caution","resnet50_pretrained_-inf_with_caution"]
        #                     }
        self.expert_dict = {
                    "without_caution":['resnet50_pretrained_cosine','resnet50_pretrained_one','resnet50_pretrained_euclidean',"resnet50_pretrained_inf","resnet50_pretrained_-inf"],
                    # "with_caution":['resnet50_pretrained_cosine_with_caution','resnet50_pretrained_one_with_caution','resnet50_pretrained_euclidean_with_caution',"resnet50_pretrained_inf_with_caution","resnet50_pretrained_-inf_with_caution"]
                    }
        self.expert_list     = ['resnet50_pretrained_cosine','resnet50_pretrained_one','resnet50_pretrained_euclidean']
        # self.expert1_results = load_expert_results(self.expert_list[0]+"_Run"+self.run_num)
        # self.expert2_results = load_expert_results(self.expert_list[1]+"_Run"+self.run_num)
        # self.expert3_results = load_expert_results(self.expert_list[2]+"_Run"+self.run_num)
        # self.expert4_results = load_expert_results(self.expert_list[3]+"_Run"+self.run_num)
        # self.expert5_results = load_expert_results(self.expert_list[4]+"_Run"+self.run_num)
        self.expert1_results = load_expert_results(self.expert_list[0])
        self.expert2_results = load_expert_results(self.expert_list[1])
        self.expert3_results = load_expert_results(self.expert_list[2])
        self.max_size = max_size
        self.test_image = test_image_path
        self.img_name = img_name
        self.messages = None

    def get_image_data(self, img_path):
        return self.resize_and_convert_to_base64(img_path)

    def resize_and_convert_to_base64(self, image_path):
        size = self.max_size
        # Open the image file
        with Image.open(image_path) as img:
            # Check the size of the image
            width, height = img.size

            # Resize if either dimension is greater than 800 pixels
            if width > size or height > size:
                # Determine the new size while maintaining the aspect ratio
                if width > height:
                    new_width = size
                    new_height = int((size / width) * height)
                else:
                    new_height = size
                    new_width = int((size / height) * width)

                # Resize the image
                img = img.resize((new_width, new_height), Image.Resampling.LANCZOS)

            # Convert the image to base64
            buffered = BytesIO()
            img.save(buffered, format="PNG")
            img_base64 = base64.b64encode(buffered.getvalue()).decode("utf-8")

            return img_base64

    def get_content(self):
        content_list = [
        {
            "type": "text",
            "text":"""You will be provided with buccal mucosa image and opinions from three indpendent medcial experts.
             You should critically analyze the reports against the image provided and generate an unbiased opininon and label the image as either "abnormal" or "normal".
             Your response should be a JSON object with the following keys:
             - "thought"
             - "output"
                        """
        },
        {
            "type": "text",
            "text":"Thought of Expert1 is as follows: "+self.expert1_results[self.img_name]["thought"]
        },
        {
            "type": "text",
            "text":"Thought of Expert2 is as follows: "+self.expert2_results[self.img_name]["thought"]
        },
        {
            "type": "text",
            "text":"Thought of Expert3 is as follows: "+self.expert3_results[self.img_name]["thought"]
        }
        # ,
        # {
        #     "type": "text",
        #     "text":"Thought of Expert4 is as follows: "+self.expert4_results[self.img_name]["thought"]
        # },
        # {
        #     "type": "text",
        #     "text":"Thought of Expert5 is as follows: "+self.expert5_results[self.img_name]["thought"]
        # }
                        ]
        return content_list
    def get_message(self):
        self.messages = [
            {"role": "user", "content": self.get_content()},
            {"role": "user", "content": [
                {
                    "type": "image_url",
                    "image_url": {
                        "url": f"data:image/jpg;base64,{self.get_image_data(self.test_image)}"
                    }
                }]
             }
        ]

In [ ]:
client = OpenAI(api_key = "your_api_key")
def predict(messages):
    response = client.chat.completions.create(
        model='gpt-4o-2024-11-20',
        messages=messages,
        temperature=0.0,
        max_tokens=200
    )
    return response.choices[0].message.content

## Prediction

### without caution

In [ ]:
moe_results = {}

In [ ]:
expert1_results = load_expert_results('resnet50_pretrained_cosine')

In [ ]:
len(expert1_results.keys())

In [ ]:
def extract_output(prediction):
    # Remove the markdown code block syntax (```json and ```)
    prediction = prediction.strip().replace('```json\n', '').replace('```', '')
    # Parse the JSON string
    prediction_dict = json.loads(prediction)
    # Return the "output" value
    return prediction_dict

In [ ]:
not_in_format_predictions = {}
for img in tqdm(list(expert1_results.keys())[:10]):
  img_path = expert1_results[img]["query_image_path"]
  moe_processor = MixtureOfExpertsProcessor(img_path,img)
  moe_processor.get_message()
  messages = moe_processor.messages
  prediction = predict(messages)
  # print(prediction)
  try:
    pred_dict = extract_output(prediction)
    moe_results[img] = {
        "query_image_path": img_path,
        "expectation": expert1_results[img]["expectation"],
        "prediction": pred_dict["output"],
        "thought": pred_dict["thought"]
    }
  except:
    print("not in format")
    not_in_format_predictions[img] = {
        "query_image_path": img_path,
        "expectation": expert1_results[img]["expectation"],
        "prediction": prediction,
        "thought": prediction
    }
  # print(moe_results)
  # break
if len(not_in_format_predictions)>0:
    moe_results['not_in_format_predictions'] = not_in_format_predictions
save_results(moe_results,"Mixture of Experts(3_experts)_test")

In [ ]:
extract_output(prediction)

In [ ]:
# moe_processor = MixtureOfExpertsProcessor(img_path,img)
# x = moe_processor.expert3_results

In [ ]:
# x.keys()

### with caution

In [ ]:
moe_results_with_caution = {}

In [ ]:
run_num = 5
expert1_results = load_expert_results('resnet50_pretrained_cosine_with_caution_Run'+str(run_num))

In [ ]:
len(expert1_results.keys())

In [ ]:
def extract_output(prediction):
    # Remove the markdown code block syntax (```json and ```)
    prediction = prediction.strip().replace('```json\n', '').replace('```', '')
    # Parse the JSON string
    prediction_dict = json.loads(prediction)
    # Return the "output" value
    return prediction_dict

In [ ]:
not_in_format_predictions = {}
not_in_format_predictions = {}
for img in tqdm(list(expert1_results.keys())):
  img_path = expert1_results[img]["query_image_path"]
  moe_processor = MixtureOfExpertsProcessor(img_path,img,with_caution=True,run_num = str(run_num))
  moe_processor.get_message()
  messages = moe_processor.messages
  prediction = predict(messages)
  # print(prediction)
  try:
    pred_dict = extract_output(prediction)
    moe_results_with_caution[img] = {
        "query_image_path": img_path,
        "expectation": expert1_results[img]["expectation"],
        "prediction": pred_dict["output"],
        "thought": pred_dict["thought"]
    }
  except:
    print("not in format")
    not_in_format_predictions[img] = {
        "query_image_path": img_path,
        "expectation": expert1_results[img]["expectation"],
        "prediction": prediction,
        "thought": prediction
    }
if len(not_in_format_predictions)>0:
    moe_results_with_caution['not_in_format_predictions'] = not_in_format_predictions
save_results(moe_results_with_caution,"Mixture of Experts with caution(5_experts)_Run"+str(run_num))

In [ ]:
moe_processor = MixtureOfExpertsProcessor(img_path,img,with_caution=True)
moe_processor.expert_list

## MOE results

In [ ]:
def load_expert_results(expert_name):
  file_name = f"Mixture of Experts/Results/{expert_name}.json"
  with open(file_name, 'r') as file:
    results = json.load(file)
  # print(f"Results loaded from {file_name}")
  return results

In [ ]:
import json
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, accuracy_score, precision_score, recall_score, f1_score
from tqdm import tqdm

# Define class mappings
class_mapping = {"normal": 0, "abnormal": 1}

# Example results dictionary (replace with your actual data)
# expert_list = ['resnet50_pretrained_cosine','resnet50_pretrained_euclidean','resnet50_pretrained_random','resnet50_pretrained_one']
results = {
    # "expert1_cosine": load_expert_results(expert_list[0]),
    # "expert2_euclidean": load_expert_results(expert_list[1]),
    # "expert3_random": load_expert_results(expert_list[2]),
    # "expert3_one": load_expert_results(expert_list[3]),
    "MoE (5 experts)": load_expert_results("Mixture of Experts(5_experts)_Run1"),
    "MoE (3 experts)": load_expert_results("Mixture of Experts(3_experts)"),
    "Moe Caution (5 experts)": load_expert_results("Mixture of Experts with caution(5_experts)_Run1")
}

# Prepare a figure for subplots
num_runs = len(results)
fig, axes = plt.subplots(2, 2, figsize=(12, 12))  # 2x2 grid

metrics = {}

# Function to extract the "output" value from the prediction string
def extract_output(prediction):
    # If the prediction is already a string (not JSON), return it directly
    if isinstance(prediction, str):
        return prediction.strip().lower()
    # If it's JSON, parse it and return the "output" value
    elif isinstance(prediction, dict):
        return prediction.get("output", "").strip().lower()
    else:
        return ""

# Process results for each run
for idx, (run_name, run_data) in enumerate(results.items()):
    # Initialize lists for actual and prediction values
    actual = []
    prediction = []
    unknown_classes = []
    metrics[run_name] = {}

    # Process each image in the run
    for img, data in tqdm(run_data.items(), desc=f"Processing images for {run_name}"):
        actual_class = data["expectation"].lower()
        pred_class = extract_output(data["prediction"])

        if pred_class not in class_mapping:
            unknown_classes.append((img, actual_class, pred_class))
            print(f"Unknown class in prediction: {pred_class}")
            continue

        else:
            pred_label = class_mapping[pred_class]
            prediction.append(pred_label)
            actual_label = class_mapping[actual_class]
            actual.append(actual_label)

    # Compute metrics
    accuracy = accuracy_score(actual, prediction)
    precision = precision_score(actual, prediction, average='binary', pos_label=class_mapping["abnormal"], zero_division=0)
    recall = recall_score(actual, prediction, average='binary', pos_label=class_mapping["abnormal"], zero_division=0)
    f1 = f1_score(actual, prediction, average='binary', pos_label=class_mapping["abnormal"], zero_division=0)

    metrics[run_name] = {
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "f1": f1,
        "unknown_classes": unknown_classes
    }

    # Compute confusion matrix
    matrix = confusion_matrix(actual, prediction, labels=[0, 1])  # Explicitly pass labels

    # Plot confusion matrix in a subplot
    ax = axes[idx // 2, idx % 2]  # Calculate the position in the 2x2 grid
    cm_display = ConfusionMatrixDisplay(confusion_matrix=matrix, display_labels=list(class_mapping.keys()))
    cm_display.plot(cmap=plt.cm.Blues, ax=ax)
    ax.set_title(
        f"{run_name}\n"
        f"Acc: {accuracy:.2f}, Prec: {precision:.2f}, Rec: {recall:.2f}, F1: {f1:.2f}"
    )

# Adjust layout and show the plot
plt.tight_layout()

plt.show()

# Print metrics
for run_name, metric_values in metrics.items():
    print(f"\nMetrics for {run_name}:")
    print(f"Accuracy: {metric_values['accuracy']:.2f}")
    print(f"Precision: {metric_values['precision']:.2f}")
    print(f"Recall: {metric_values['recall']:.2f}")
    print(f"F1 Score: {metric_values['f1']:.2f}")
    if metric_values["unknown_classes"]:
        print(f"Unknown classes: {metric_values['unknown_classes']}")

# MOE without caution results 5 experts

In [ ]:
import json
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, accuracy_score, precision_score, recall_score, f1_score
from tqdm import tqdm

# Define class mappings
class_mapping = {"normal": 0, "abnormal": 1}

# Example results dictionary (replace with your actual data)
expert_list = ['resnet50_pretrained_cosine','resnet50_pretrained_euclidean','resnet50_pretrained_random','resnet50_pretrained_one',"resnet50_pretrained_inf","resnet50_pretrained_-inf","Mixture of Experts(5_experts)"]

results = {
    "random": load_expert_results(expert_list[2]),
    "cosine": load_expert_results(expert_list[0]),
    "one": load_expert_results(expert_list[3]),
    "euclidean": load_expert_results(expert_list[1]),
    "inf": load_expert_results(expert_list[4]),
    "-inf": load_expert_results(expert_list[5]),
    "moe_results": load_expert_results(expert_list[6])
}

# Prepare a figure for subplots
num_runs = len(results)
fig, axes = plt.subplots(3, 3, figsize=(18, 18))  # 3x3 grid

metrics = {}

# Function to extract the "output" value from the prediction string
def extract_output(prediction):
    # If the prediction is already a string (not JSON), return it directly
    if isinstance(prediction, str):
        return prediction.strip().lower()
    # If it's JSON, parse it and return the "output" value
    elif isinstance(prediction, dict):
        return prediction.get("output", "").strip().lower()
    else:
        return ""

# Process results for each run
for idx, (run_name, run_data) in enumerate(results.items()):
    # Initialize lists for actual and prediction values
    actual = []
    prediction = []
    unknown_classes = []
    metrics[run_name] = {}

    # Process each image in the run
    for img, data in tqdm(run_data.items(), desc=f"Processing images for {run_name}"):
        actual_class = data["expectation"].lower()
        pred_class = extract_output(data["prediction"])

        if pred_class not in class_mapping:
            unknown_classes.append((img, actual_class, pred_class))
            print(f"Unknown class in prediction: {pred_class}")
            continue

        else:
            pred_label = class_mapping[pred_class]
            prediction.append(pred_label)
            actual_label = class_mapping[actual_class]
            actual.append(actual_label)

    # Compute metrics
    accuracy = accuracy_score(actual, prediction)
    precision = precision_score(actual, prediction, average='binary', pos_label=class_mapping["abnormal"], zero_division=0)
    recall = recall_score(actual, prediction, average='binary', pos_label=class_mapping["abnormal"], zero_division=0)
    f1 = f1_score(actual, prediction, average='binary', pos_label=class_mapping["abnormal"], zero_division=0)

    metrics[run_name] = {
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "f1": f1,
        "unknown_classes": unknown_classes
    }

    # Compute confusion matrix
    matrix = confusion_matrix(actual, prediction, labels=[0, 1])  # Explicitly pass labels

    # Plot confusion matrix in a subplot
    ax = axes[idx // 3, idx % 3]  # Calculate the position in the 3x3 grid
    cm_display = ConfusionMatrixDisplay(confusion_matrix=matrix, display_labels=list(class_mapping.keys()))
    cm_display.plot(cmap=plt.cm.Blues, ax=ax)
    ax.set_title(
        f"{run_name}\n"
        f"Acc: {accuracy:.2f}, Prec: {precision:.2f}, Rec: {recall:.2f}, F1: {f1:.2f}"
    )

# Adjust layout and show the plot
plt.tight_layout()
plt.show()

# Print metrics
for run_name, metric_values in metrics.items():
    print(f"\nMetrics for {run_name}:")
    print(f"Accuracy: {metric_values['accuracy']:.2f}")
    print(f"Precision: {metric_values['precision']:.2f}")
    print(f"Recall: {metric_values['recall']:.2f}")
    print(f"F1 Score: {metric_values['f1']:.2f}")
    if metric_values["unknown_classes"]:
        print(f"Unknown classes: {metric_values['unknown_classes']}")

# MOE with caution results 5 experts

In [ ]:
import json
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, accuracy_score, precision_score, recall_score, f1_score
from tqdm import tqdm

# Define class mappings
class_mapping = {"normal": 0, "abnormal": 1}

# Example results dictionary (replace with your actual data)
expert_list = ['resnet50_pretrained_cosine_with_caution','resnet50_pretrained_euclidean_with_caution','resnet50_pretrained_random','resnet50_pretrained_one_with_caution',"resnet50_pretrained_inf_with_caution","resnet50_pretrained_-inf_with_caution","Mixture of Experts with caution(5_experts)"]

results = {
    "random": load_expert_results(expert_list[2]),
    "cosine_with_caution": load_expert_results(expert_list[0]),
    "one_with_caution": load_expert_results(expert_list[3]),
    "euclidean_with_caution": load_expert_results(expert_list[1]),
    "inf_with_caution": load_expert_results(expert_list[4]),
    "-inf_with_caution": load_expert_results(expert_list[5]),
    "moe_results_with_caution": load_expert_results(expert_list[6])
}

# Prepare a figure for subplots
num_runs = len(results)
fig, axes = plt.subplots(3, 3, figsize=(18, 18))  # 3x3 grid

metrics = {}

# Function to extract the "output" value from the prediction string
def extract_output(prediction):
    # If the prediction is already a string (not JSON), return it directly
    if isinstance(prediction, str):
        return prediction.strip().lower()
    # If it's JSON, parse it and return the "output" value
    elif isinstance(prediction, dict):
        return prediction.get("output", "").strip().lower()
    else:
        return ""

# Process results for each run
for idx, (run_name, run_data) in enumerate(results.items()):
    # Initialize lists for actual and prediction values
    actual = []
    prediction = []
    unknown_classes = []
    metrics[run_name] = {}

    # Process each image in the run
    for img, data in tqdm(run_data.items(), desc=f"Processing images for {run_name}"):
        actual_class = data["expectation"].lower()
        pred_class = extract_output(data["prediction"])

        if pred_class not in class_mapping:
            unknown_classes.append((img, actual_class, pred_class))
            print(f"Unknown class in prediction: {pred_class}")
            continue

        else:
            pred_label = class_mapping[pred_class]
            prediction.append(pred_label)
            actual_label = class_mapping[actual_class]
            actual.append(actual_label)

    # Compute metrics
    accuracy = accuracy_score(actual, prediction)
    precision = precision_score(actual, prediction, average='binary', pos_label=class_mapping["abnormal"], zero_division=0)
    recall = recall_score(actual, prediction, average='binary', pos_label=class_mapping["abnormal"], zero_division=0)
    f1 = f1_score(actual, prediction, average='binary', pos_label=class_mapping["abnormal"], zero_division=0)

    metrics[run_name] = {
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "f1": f1,
        "unknown_classes": unknown_classes
    }

    # Compute confusion matrix
    matrix = confusion_matrix(actual, prediction, labels=[0, 1])  # Explicitly pass labels

    # Plot confusion matrix in a subplot
    ax = axes[idx // 3, idx % 3]  # Calculate the position in the 3x3 grid
    cm_display = ConfusionMatrixDisplay(confusion_matrix=matrix, display_labels=list(class_mapping.keys()))
    cm_display.plot(cmap=plt.cm.Blues, ax=ax)
    ax.set_title(
        f"{run_name}\n"
        f"Acc: {accuracy:.2f}, Prec: {precision:.2f}, Rec: {recall:.2f}, F1: {f1:.2f}"
    )

# Adjust layout and show the plot
plt.tight_layout()
plt.show()

# Print metrics
for run_name, metric_values in metrics.items():
    print(f"\nMetrics for {run_name}:")
    print(f"Accuracy: {metric_values['accuracy']:.2f}")
    print(f"Precision: {metric_values['precision']:.2f}")
    print(f"Recall: {metric_values['recall']:.2f}")
    print(f"F1 Score: {metric_values['f1']:.2f}")
    if metric_values["unknown_classes"]:
        print(f"Unknown classes: {metric_values['unknown_classes']}")

In [ ]:
def extract_output(prediction):
    # Remove the markdown code block syntax (```json and ```)
    prediction = prediction.strip().replace('```json\n', '').replace('```', '')
    # Parse the JSON string
    prediction_dict = json.loads(prediction)
    # Return the "output" value
    return prediction_dict

In [ ]:
# = load_expert_results("resnet50_pretrained_cosine_with_caution")
with open(f"Result.json", 'r') as file:
    cosine_expert = json.load(file)['Run1']
resnet50_pretrained_cosine_with_caution = {}
not_in_format_predictions = {}
for img in tqdm(list(cosine_expert.keys())):
  img_path = cosine_expert[img]["query_image_path"]
  prediction = cosine_expert[img]["prediction"]
  try:
    pred_dict = extract_output(prediction)
    # print(prediction)
    # print(type(pred_dict))
    resnet50_pretrained_cosine_with_caution[img] = {
        "query_image_path": img_path,
        "expectation": cosine_expert[img]["expectation"],
        "prediction": pred_dict["output"],
        "thought": pred_dict["thought"]
    }
  except Exception as e:
    # print(e)
    # break
    print("not in format")
    not_in_format_predictions[img] = {
        "query_image_path": img_path,
        "expectation": cosine_expert[img]["expectation"],
        "prediction": prediction,
        "thought": prediction
    }

if len(not_in_format_predictions)>0:
    resnet50_pretrained_cosine_with_caution['not_in_format_predictions'] = not_in_format_predictions
# save_results(moe_results,"Mixture of Experts(5_experts)")